In [3]:
import requests
import polars as pl
from datetime import datetime, timedelta
import multiprocessing
import config
API_KEY = config.polygon_api1

In [3]:
#Get dataframe of all tickers
constituents = pl.read_csv("../Data/sp500/Constituents.csv", infer_schema_length=False)

#Create list of individual stocks from names from dataframe
stockComponents=[]
for k in range(len(constituents)):
    stockComponents.append(constituents[k,0])

In [4]:
for k in range(len(stockComponents)):
    ticker = stockComponents[k]
    timespan = "day"
    start_date = datetime(2021, 10, 7)
    end_date = datetime(2023, 10, 2)
    all_data = []

    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/{timespan}/{start_date.date()}/{end_date.date()}?adjusted=true&sort=asc&limit=50000&apiKey={API_KEY}"
    response = requests.get(url)
    data = response.json()
    if "results" in data:
        all_data.extend(data["results"])
    else:
        print(data)
    df = pl.DataFrame(all_data)
    df = df.rename({"v":"volume", "vw":"weightedVolume", "o":"open", "c":"close", "h":"high", "l":"low", "t":"time", "n":"numberOfTrades"})
    df.write_csv('../Data/SP500/dailyHist2000/'+stockComponents[k]+'.csv')

In [4]:
def getStockData(stockComponent):
    ticker = stockComponent
    timespan = "minute"
    start_date = datetime(2021, 3, 15)
    end_date = datetime(2023, 10, 10)
    batch_size = 60
    current_date = start_date
    all_data = []

    while current_date < end_date:
        batch_end_date = current_date + timedelta(days=batch_size)
        if batch_end_date > end_date:
            batch_end_date = end_date
        url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/{timespan}/{current_date.date()}/{batch_end_date.date()}?adjusted=true&sort=asc&limit=50000&apiKey={API_KEY}"
        response = requests.get(url)
        data = response.json()
        if "results" in data:
            all_data.extend(data["results"])
        else:
            print(data)
        current_date = batch_end_date + timedelta(days=1)
    df = pl.DataFrame(all_data)
    df = df.rename({"v":"volume", "vw":"weighted volume", "o":"open", "c":"close", "h":"high", "l":"low", "t":"time", "n":"number of trades"})
    df.write_csv('../Data/SP500/minuteHist2021/afterHours/'+ticker+'.csv')

In [5]:
for k in range(len(stockComponents)):
    ticker = stockComponents[k]
    timespan = "minute"
    start_date = datetime(2021, 3, 15)
    end_date = datetime(2023, 11, 3)
    batch_size = 60
    current_date = start_date
    all_data = []

    while current_date < end_date:
        batch_end_date = current_date + timedelta(days=batch_size)
        if batch_end_date > end_date:
            batch_end_date = end_date
        url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/{timespan}/{current_date.date()}/{batch_end_date.date()}?adjusted=true&sort=asc&limit=50000&apiKey={API_KEY}"
        response = requests.get(url)
        data = response.json()
        if "results" in data:
            all_data.extend(data["results"])
        else:
            print(data)
        current_date = batch_end_date + timedelta(days=1)
    df = pl.DataFrame(all_data)
    df = df.rename({"v":"volume", "vw":"weighted volume", "o":"open", "c":"close", "h":"high", "l":"low", "t":"time", "n":"number of trades"})
    df.write_csv('../Data/SP500/minuteHist2021/afterHours/'+ticker+'.csv')

{'ticker': 'ABC', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '23ad45d9bf2f3508a2fc1fbd2cdaf352'}
{'ticker': 'BALL', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '925d271bea9c4d2af34043523deccc70'}
{'ticker': 'BALL', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': 'afc378523aecd61d50b8661115b64b6b'}
{'ticker': 'BALL', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '0bbe64a616cc3dc4f6468ff10ebcb7de'}
{'ticker': 'BALL', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '853be059ae8181c6c938c225199e25e5'}
{'ticker': 'BALL', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '6029c3e3fc97c78d703d57cae470ce3e'}
{'ticker': 'BALL', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '7f550eafb654f4cd975cbac21a9c700e'}
{'ticker': 'BBWI', 'queryCou

In [8]:
df = pl.DataFrame(all_data)

In [7]:
#Sorting out trading hours data accounting for daylight savings time
for k in range(len(stockComponents)):
    current=pl.read_csv('../Data/SP500/minuteHist2021/afterHours/'+stockComponents[k]+'.csv')

    new=current.filter((pl.col("time")%86400000) < 72000000)
    new2=new.filter((pl.col("time")%86400000) >= 48600000)

    final1=new2.filter(pl.col("time") >= 1678590153000)

    new3=new2.filter(pl.col("time") <= 1667703753000)
    final2=new3.filter(pl.col("time") >= 1647140553000)

    final3=new2.filter(pl.col("time") <= 1636254153000)

    new4=current.filter((pl.col("time")%86400000) < 75600000)
    new5=new4.filter((pl.col("time")%86400000) >= 52200000)

    new6=new5.filter(pl.col("time") <= 1647140553000)
    final4=new6.filter(pl.col("time") >= 1636254153000)

    new7=new5.filter(pl.col("time") <= 1678590153000)
    final5=new7.filter(pl.col("time") >= 1667703753000)

    final=pl.concat([final3,final4,final2,final5,final1])
    final.write_csv('../Data/SP500/minuteHist2021/tradingHours/'+stockComponents[k]+'.csv')